In [1]:
import importlib
import pandas as pd
import numpy as np
import re

In [2]:
from functions.get_mon_data import *
from functions.functions_dataframe import * 
from functions.functions_shared import *
from parameters.filters_lists import *
from parameters.route import *
from parameters.regs import *

In [3]:
"""
Завантажити бази шкіл і садків МОН з data.gov.ua
"""
expdata_sc = pandas.read_csv('data/mon_data/expdata_sc.csv', sep = ';')
expdata_dnz = pandas.read_csv('data/mon_data/expdata_dnz.csv', sep = ';')

In [4]:
"""
Фільтрування бази МОН шкільних закладів за довжиною ЄДРПОУ коду (залишити тільки 7-8 значні коди, 
інші вважаються помилковими)
"""
expdata_sc = expdata_sc.fillna(0)
expdata_sc['edrpou'] = expdata_sc['edrpou'].astype(np.int64)
mask = (expdata_sc['edrpou'].astype(str).str.len() == 7) | (expdata_sc['edrpou'].astype(str).str.len() == 8)
expdata_sc = expdata_sc.loc[mask].sort_values(by=['edrpou'])

In [5]:
"""
Фільтрування бази МОН дитячих садків за довжиною ЄДРПОУ коду (залишити тільки 7-8 значні коди, 
інші вважаються помилковими)
"""
expdata_dnz['edrpou'] = pd.to_numeric(expdata_dnz['edrpou'],errors='coerce')
expdata_dnz = expdata_dnz.fillna(0)
expdata_dnz['edrpou'] = expdata_dnz['edrpou'].astype(np.int64)
mask = (expdata_dnz['edrpou'].astype(str).str.len() == 7) | (expdata_dnz['edrpou'].astype(str).str.len() == 8)
expdata_dnz = expdata_dnz.loc[mask].sort_values(by=['edrpou'])

In [6]:
"""
Об'єднати датафрейми садків і шкіл в один датафрейм
"""
frames = [expdata_dnz, expdata_sc]
df_mon_data = pd.concat(frames)

In [7]:
"""
З бази ProZorro df_raw створений перелік ЄДРПОУ управлінь освіти, які трапляються в списку МОН
"""
df_prozorro = pandas.read_csv('data/TrainingProZorroForSchoolProject20162019.csv', sep = ';', low_memory = False)
filter_keywords_exclude = ["департамент", "управлін.*освіт", "відділ.*освіт"]
df_raw_upravlin_osvity = df_prozorro[df_prozorro['Организатор'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords_exclude)), case = False, na = False, regex = True)]
edrpou_upravlin_osvity_prozorro = df_raw_upravlin_osvity['IDOrganizator'].unique().tolist()

/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


In [8]:
"""
Виключити з переліку бази МОН df_mon_data освітні департаменти, управління освіти
"""
mask = (df_mon_data['edrpou'].isin(edrpou_upravlin_osvity_prozorro))
df_mon_data_bez_upravlin_osvity = df_mon_data.loc[~mask]

In [ ]:
"""
Зберегти датафрейм МОН навчальних закладів, з якого виделані департаменти, управління, відділи
"""
df_mon_data_bez_upravlin_osvity.to_csv('data/df_bez_upravlin_osvity.csv', sep=';')

### Створити список власних назв шкіл і садків

In [117]:
"""
Залишити з датафрейму df_mon_data_bez_upravlin_osvity одну колонку - з назвами шкіл і садків 
"""
df_university_names = df_mon_data_bez_upravlin_osvity.loc[:, df_mon_data_bez_upravlin_osvity.columns.intersection(['university_name'])]

In [118]:
"""
Замінити лапки типу «» на " 
"""
df_university_names.university_name = df_university_names.university_name.str.replace('«|»', '"')

In [119]:
"""
Створити колонку "Назва" зі знайденими у лапках словами
"""
df_university_names_filtered = naming_one_column(df_university_names, ['university_name'], reg_double_quotes)

In [120]:
"""
Видалити пусті рядки
"""
df_university_names_filtered.dropna(inplace=True)

In [121]:
searchfor = ['№', 'заклад']
df_names_clean = df_university_names_filtered[~df_university_names_filtered["Назва"].str.contains('|'.join(searchfor), case = False)]

In [122]:
"""
Видалити пусті рядки
"""
df_names_clean.dropna(inplace=True)

/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [123]:
"""
Видалити символи "()
"""
df_names_clean['Назва'] = df_names_clean['Назва'].str.replace('"', '')
df_names_clean['Назва'] = df_names_clean['Назва'].str.replace(')', '')
df_names_clean['Назва'] = df_names_clean['Назва'].str.replace('(', '')

/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pa

In [125]:
"""
Зберегти в окремий файл df_name_list.csv
"""
df_names_clean.to_csv('data/df_names_clean.csv', sep=';')

In [126]:
df_names_clean[df_names_clean['Назва'].str.contains('Полтавської міської ради')]

,university_name,Назва
5063,Полтавський дошкільний навчальний заклад № 21 ...,Полтавської міської ради Полтавської області


In [133]:
df_names_clean[df_names_clean['Назва'] == 'Невідомо']

,university_name,Назва
